My initial idea, creating my own stuff semi based on pytorch before totaly torch'ing it

In [23]:
import torch
from natsort import natsorted
import json

from pathlib import Path
root = Path().resolve().parent

folder = root / '1 DATA + CLEANUP' / 'game_files' / 'train tokenized'
idx_folder = root / '1 DATA + CLEANUP' / 'game_files' / 'train maps'

# dont know if to use normal Dataset or IterableDataset from pytorch

class my_train_Dataset(): # supposed to be used with train files and maps

    def __init__(self, in_folder = folder, idx_folder = idx_folder):

        self.in_folder = in_folder  # folder with chunk files
        self.idx_folder = idx_folder    # folder with map files
        self.gpchunk = []    # number of games per file / map

        # chunk and map variables
        self.current_chunk = None
        self.current_map = None
        self.n_chunks = None
        self.current_chunk_data = None
        self.current_map_data = None

        # functions to run on init
        self.game_count()
        self.cm_tuple()

    def game_count(self): # number of games in each chunk
        if 'train' in str(self.in_folder):
            for i in range(len(self.in_chunks)): # pre-computed
                n = 2_500_000
                if i == 9: n = 2456165
                if i == 19: n = 2451750
                if i == 29: n = 2450871
                self.gpchunk.append([i, n])

    def cm_tuple(self):   # chunk-maps tuples
        self.in_chunks = natsorted([f for f in self.in_folder.iterdir()])   # list of chunk paths
        self.n_chunks = len(self.in_chunks)
        self.maps = natsorted([f for f in self.idx_folder.iterdir()])   # list of map paths
        if len(self.in_chunks) != len(self.maps):   # shouldnt need to be ran but just in case
            raise ValueError("Number of input chunks(files) and index files do not match.")
        else:
            self.chunks_maps = {i:t for i, t in enumerate(zip(self.in_chunks, self.maps))} # (chunk, map)

    def get_chunk(self, chunk_idx): # helper function to load chunk and map data
        if chunk_idx >= len(self.chunks_maps) or chunk_idx < 0:
            raise IndexError("Index out of range.")
        chunk, map = self.chunks_maps[chunk_idx]
        if chunk == self.current_chunk and map == self.current_map: # dont reload if same chunk
            return self.current_chunk_data, self.current_map_data
        self.current_chunk = chunk  # chunk file path
        self.current_map = map  # map file path
        self.current_chunk_data = torch.load(chunk)   # chunk data (flat tensor of ~2.5M tokenized games)
        with open(map) as m: self.current_map_data = json.load(m)  # map data (list of tuples with game indices)
        return self.current_chunk_data, self.current_map_data

    def __len__(self):  # total number of games in all chunks (idk, pytorch says it wants it)
        return 2_500_000 * 33 + 2456165 + 2451750 + 2450871
    
    def __getitem__(self, chunk_idx, game_idx): # get game from chunk
        self.get_chunk(chunk_idx)
        return self.current_chunk_data[self.current_map_data[game_idx][0]:self.current_map_data[game_idx][1]] # game_idx'th game from chunk_idx'th chunk

#############################

class my_Sampler(): # sampler for train data

    def __init__(self, dataset, shuffle_chunks=True, shuffle_games=True):
        self.chunk_order = torch.arange(len(dataset.self.n_chunks))
        self.game_order = torch.arange(2_500_000) # 2.5M games per chunk (tho some arent... idk yet)
        self.shuffle_chunks = shuffle_chunks
        self.shuffle_games = shuffle_games
        self.shuffle()

    def shuffle(self):
        if self.shuffle_chunks:
            self.chunks_order = torch.randperm(len(dataset.self.n_chunks))
        if self.shuffle_games: # idk how to make the order dependent on the chunk size, some are smaller
            self.games_order = torch.randperm(2_500_000).to_list()

    def __iter__(self):
        for chunk_idx in self.chunk_order:
            for game_idx in self.game_order:
                yield chunk_idx, game_idx

##########################

from torch.nn.utils.rnn import pad_sequence

def my_collate_fn(batch):
    games = [tensor for tensor in batch]  # list of game tensors
    games = pad_sequence(games, batch_first=True, padding_value=0)  # pad to max length in batch
    return games

---

---

change after help from AI

In [123]:
import torch
import math
from natsort import natsorted
import random
import json
#import dataloader

from torch.utils.data import IterableDataset, DataLoader
from pathlib import Path
root = Path().resolve().parent

c_folder = root / '1 DATA + CLEANUP' / 'game_files' / 'train tokenized'
map_folder = root / '1 DATA + CLEANUP' / 'game_files' / 'train maps'

class my_Dataset(IterableDataset):

    def __init__(self, in_folder, idx_folder, shuffle=True):
        self.shuffle = shuffle
        # chunk and map paths
        self.in_folder = in_folder
        self.idx_folder = idx_folder
        # chunk and map variables
        self.chunks_maps = None
        # functions to run on init
        self.cm_tuple()

    def cm_tuple(self):   # chunk-maps tuples
        in_chunks = natsorted([f for f in self.in_folder.iterdir() if '.pt' in str(f)])   # list of chunk paths (without game_counts)
        self.n_chunks = len(in_chunks)
        maps = natsorted([f for f in self.idx_folder.iterdir()])   # list of map paths
        if len(in_chunks) != len(maps):   # shouldnt need to be ran but just in case
            raise ValueError("Number of input chunks(files) and index files do not match.")
        else:
            self.chunks_maps = list(zip(in_chunks, maps)) # (chunk, map)

    def __iter__(self):
        # --- Worker information and workload splitting ---
        worker_info = torch.utils.data.get_worker_info()
        num_workers = worker_info.num_workers if worker_info is not None else 1
        worker_id = worker_info.id if worker_info is not None else 0

        # --- Determine the chunks for this worker ---
        all_chunk_map_pairs = list(self.chunks_maps) # Get the original list (make copy if needed)
        num_chunks = len(all_chunk_map_pairs)

        if num_chunks == 0:
            return # Stop iteration if dataset is empty

        # Calculate the range of chunks this worker should process (based on original order)
        per_worker = int(math.ceil(num_chunks / float(num_workers)))
        iter_start = worker_id * per_worker
        iter_end = min(iter_start + per_worker, num_chunks)

        # Get the subset of chunk paths assigned to this worker
        assigned_chunk_map_pairs = all_chunk_map_pairs[iter_start:iter_end] # Slice FIRST

        # Shuffle the subset assigned to this worker (optional)
        if self.shuffle:
            random.shuffle(assigned_chunk_map_pairs) # Shuffle the worker's ASSIGNED portion SECOND

        # --- Process assigned chunks ---
        print(f"[Worker {worker_id}] Processing {len(assigned_chunk_map_pairs)} chunks (indices from original list: {iter_start} to {iter_end-1}).") # Optional debug print

        for chunk_path, map_path in assigned_chunk_map_pairs:
            try:
                # Load data for the current chunk
                # Consider adding more robust file existence checks if needed
                if not chunk_path.exists() or not map_path.exists():
                     print(f"[Worker {worker_id}] Warning: File missing for chunk {chunk_path.name} or map {map_path.name}. Skipping.")
                     continue
                     
                # print(f"[Worker {worker_id}] Loading chunk: {chunk_path.name}") # Optional debug print
                chunk_data = torch.load(chunk_path)
                with open(map_path, 'r') as f:
                    map_data = json.load(f) # Should be a list of [start, end] pairs

                num_games_in_chunk = len(map_data)
                if num_games_in_chunk == 0:
                    # print(f"[Worker {worker_id}] Warning: Chunk {chunk_path.name} has 0 games in map. Skipping.") # Optional debug print
                    continue # Skip empty chunks

                # Generate local game indices (0 to num_games-1)
                local_game_indices = list(range(num_games_in_chunk))

                # Shuffle local game indices within the chunk (optional)
                if self.shuffle:
                    random.shuffle(local_game_indices)

                # Yield games from this chunk based on shuffled local indices
                for game_idx in local_game_indices:
                    try:
                        start, end = map_data[game_idx]
                        yield chunk_data[start:end]
                    except IndexError:
                         print(f"[Worker {worker_id}] Error: Game index {game_idx} out of bounds for map {map_path.name} (len={num_games_in_chunk}). Skipping game.")
                    except Exception as game_e:
                         print(f"[Worker {worker_id}] Error yielding game {game_idx} from chunk {chunk_path.name}: {game_e}")
                         
                # Optional: Explicitly free memory for the large chunk data
                del chunk_data
                del map_data
                del local_game_indices

            except FileNotFoundError:
                 print(f"[Worker {worker_id}] Error: File not found for chunk {chunk_path} or map {map_path}. Skipping chunk.")
            except Exception as e:
                # Log error or decide how to handle other corrupted files/chunks
                print(f"[Worker {worker_id}] Error processing chunk {chunk_path.name}: {e}")
                # Optionally 'del chunk_data' etc. here too in case they exist
                continue # Skip to the next chunk

##########################

from torch.nn.utils.rnn import pad_sequence

def my_collate_fn(batch):
    games = [tensor for tensor in batch]  # list of game tensors
    games = pad_sequence(games, batch_first=True, padding_value=0)  # pad to max length in batch
    return games

In [124]:
dataset = my_Dataset(c_folder, map_folder)

In [114]:
#--- How to use it ---
# dataset = ChessIterableDataset(chunk_folder='/path/to/chunks', map_folder='/path/to/maps')
# # IMPORTANT: collate_fn is still needed for padding batches!
# dataloader = torch.utils.data.DataLoader(
#     dataset,
#     batch_size=32,
#     num_workers=4, # Example: Use 4 worker processes
#     collate_fn=my_collate_fn, # Your function to pad sequences in a batch
#     pin_memory=True # Often good practice if using GPU
# )

# for epoch in range(num_epochs):
#     print(f"\n--- Epoch {epoch+1} ---")
#     for i, batch in enumerate(dataloader):
#         # Your training step here with the batch
#         print(f"Processing batch {i}, shape: {batch.shape}") # Example
#         # model(batch) ... etc


In [129]:
dataloader = DataLoader(dataset, batch_size=4, num_workers=2, collate_fn=my_collate_fn)#, pin_memory=True)

In [ ]:
# runs but need gpu
for epoch in range(1):
    print(f"\n--- Epoch {epoch+1} ---")
    for i, batch in enumerate(dataloader):
        # Your training step here with the batch
        print(f"Processing batch {i}, shape: {batch.shape}") # Example
        # model(batch) ... etc